In [220]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('..')

In [221]:
data = pd.read_csv('../data/interim/all_train_data.csv')

In [222]:
data.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [223]:
pct = np.percentile(data.loc[:, 'area_percentage'].fillna(np.mean(data.loc[:, 'area_percentage'])), 95)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'area_percentage'] < pct]
print(data.shape)

16.0
(260601, 39)
(247044, 39)


In [224]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np
#height_percentage
# Function to create interaction terms
def create_interaction_terms(X):
    # Assuming X is a dataframe with the columns geo_level_1_id, geo_level_2_id, geo_level_3_id
    X_new = X.copy()
    #X_new['areaHeight'] = X_new['height_percentage'] / X_new['area_percentage']
    X_new['geo1times2'] = X_new['geo_level_1_id'] * X_new['geo_level_2_id']
    X_new['geo1times3'] = X_new['geo_level_1_id'] * X_new['geo_level_3_id']
    X_new['geo2times3'] = X_new['geo_level_2_id'] * X_new['geo_level_3_id']
    X_new['geoall'] = X_new['geo_level_2_id'] * X_new['geo_level_3_id'] * X_new['geo_level_1_id']
    return X_new[['geo1times2', 'geo1times3', 'geo2times3', 'geoall']]#, 'areaHeight']]  # Return the interaction terms only

# Creating the FunctionTransformer
interaction_transformer = FunctionTransformer(create_interaction_terms)


In [337]:
numerical_df = data.select_dtypes(exclude=['object'])
numerical_df.describe()
counts = numerical_df['geo_level_3_id'].value_counts()

min_count = counts.min()
least_frequent_numbers = counts[counts == min_count]
print(least_frequent_numbers)
print(least_frequent_numbers.count())


geo_level_3_id
6536     1
6776     1
5812     1
10748    1
4396     1
        ..
8635     1
12567    1
5102     1
4178     1
3085     1
Name: count, Length: 963, dtype: int64
963


In [226]:
from scipy.stats import skew

def get_right_skewed_columns(df, skew_threshold=0.5):
    """
    Returns the names of columns that are right-skewed based on the skewness value, excluding binary columns.
    
    Parameters:
    - df: The input DataFrame (numerical columns only).
    - skew_threshold: The skewness threshold above which a column is considered right-skewed (default is 0.5).
    
    Returns:
    - List of column names that are right-skewed.
    """
    right_skewed_columns = []
    
    # Iterate through each column in the dataframe
    for col in df.columns:
        # Check if the column has more than 2 unique values (to avoid binary columns)
        if df[col].nunique() > 2:
            # Calculate skewness for each column
            col_skewness = skew(df[col].dropna())  # Drop NaN values to avoid issues
            
            # Check if the skewness is above the specified threshold (indicating right-skewness)
            if col_skewness > skew_threshold:
                right_skewed_columns.append(col)
    
    return right_skewed_columns


In [227]:
# # Select numerical columns
# numerical_df = data.select_dtypes(exclude=['object'])

# # Get the right-skewed columns
right_skewed_cols = get_right_skewed_columns(numerical_df)

print("Right-skewed columns:", right_skewed_cols)


Right-skewed columns: ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']


In [228]:
# import numpy as np

# def apply_log_transformation(df, columns):
#     """
#     Applies log transformation to the specified columns of the DataFrame.
    
#     Parameters:
#     - df: The input DataFrame.
#     - columns: List of column names to apply the log transformation on.
    
#     Returns:
#     - DataFrame with log-transformed columns.
#     """
    
#     df_transformed = df.copy()
    
#     # Apply log transformation to each specified column
#     for col in columns:
#         # Add a small constant to avoid log(0) and handle any zeros or negatives
#         df_transformed[col] = np.log1p([col])
    
#     return df_transformed


In [229]:
# Get the right-skewed columns (excluding binary columns)
#right_skewed_cols = get_right_skewed_columns(numerical_df)
#numerical_df = apply_log_transformation(numerical_df, right_skewed_cols)
#data[right_skewed_cols] = numerical_df[right_skewed_cols]

In [230]:
#pct = np.percentile(data.loc[:, 'age'].fillna(np.mean(data.loc[:, 'age'])), 99)
#print(pct)
#print(data.shape)
#data = data.loc[data.loc[:, 'age'] < pct]
#print(data.shape)

In [231]:
def log_transform(X):
    # Apply log1p transformation (log(1 + x)) to avoid issues with zero values
    return np.log1p(X)

# Create a FunctionTransformer for log transformation
log_transformer = FunctionTransformer(log_transform)


In [232]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Custom transformer for the age-based transformation
class AgeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, age_column='age'):
        self.age_column = age_column
        self.percentile_ = None

    def fit(self, X, y=None):
        # Calculate the 99th percentile of the 'age' column and store it
        self.percentile_ = np.percentile(X[self.age_column].fillna(np.mean(X[self.age_column])), 99)
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        # Add a new 'old' column to indicate if the age exceeds the 99th percentile
        X_copy['old'] = np.where(X_copy[self.age_column] >= self.percentile_, 1, 0)
        
        # Cap the age to 100 where the 'old' column is 1
        X_copy.loc[X_copy['old'] == 1, self.age_column] = 100
        
        return X_copy


In [233]:
x = data.drop(columns=['damage_grade'])
y = data.damage_grade

In [234]:
y = y.replace({1: 0, 2: 1, 3: 2})

In [235]:
y

0         2
1         1
2         2
3         1
4         2
         ..
260596    1
260597    2
260598    2
260599    1
260600    2
Name: damage_grade, Length: 247044, dtype: int64

In [236]:
y.value_counts()

damage_grade
1    141177
2     84323
0     21544
Name: count, dtype: int64

In [237]:
#x.old.value_counts()

In [238]:
#numerical_df = data.select_dtypes(exclude=['object'])
#categorical_df = data.select_dtypes(include=['object'])

In [239]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in x.columns if  x[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in x.columns if x[cname].dtype in ['int32', 'int64', 'float64']]

In [240]:
categorical_cols

['land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [241]:
#plt.hist(numerical_df.age)

In [242]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler, MinMaxScaler

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42,  stratify=y) #shuffle=True,

In [243]:
y_train.value_counts()

damage_grade
1    98823
2    59026
0    15081
Name: count, dtype: int64

In [244]:
y_test.value_counts()

damage_grade
1    42354
2    25297
0     6463
Name: count, dtype: int64

In [245]:
from category_encoders import BaseNEncoder

In [246]:
#from src.eda_first import summarize_dataframe
#summarize_dataframe(X_train)

In [247]:
#create numerical transformer

numerical_transformer = Pipeline([('imputer', SimpleImputer(strategy='mean'))])

#create categorical transformer
#categorical_transformer = Pipeline(steps=[ ('imputer', SimpleImputer(strategy='most_frequent')),
#                                            ('onehot', OneHotEncoder(handle_unknown='ignore'))
#                                            ])

base_encoder_columns = ['land_surface_condition','geo_level_1_id', 'geo_level_2_id','geo_level_3_id',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

base_encoder = Pipeline(steps=[
    ('base_encoder', BaseNEncoder(cols=base_encoder_columns, base=3))
])

age_transformer = Pipeline(steps=[
    ('age_transform', AgeTransformer(age_column='age'))  # Apply age transformation
])

In [248]:
# Combine the transformations using ColumnTransformer
#preprocessor = ColumnTransformer(transformers=[
#    ('base_name', base_encoder, base_encoder_columns)])  # TargetEncoder for 'town'
#    ('num', numerical_transformer, numerical_cols)])

In [249]:
# Updated ColumnTransformer with the log transformer
preprocessor = ColumnTransformer(transformers=[
    ('base_name', base_encoder, base_encoder_columns),  # BaseNEncoder for categorical columns
    ('age_transform', age_transformer, ['age']),  # Custom transformer for the 'age' column
    ('num', 'passthrough', numerical_cols),  # Pass numerical columns through without transformation
    ('interaction', interaction_transformer, ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'height_percentage', 'area_percentage']),  # Interaction terms
    ('log_transform', log_transformer, right_skewed_cols)  # Apply log transformation to specified columns
])

In [304]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBClassifier
import numpy as np

# Split the data into training and validation sets for cross-validation
X_train_full, X_val, y_train_full, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Model 1: Binary Classification (0 vs Others)
xgb_binary = XGBClassifier(
    n_estimators=500,
    learning_rate=0.2669112505018992,
    max_depth=5,
    random_state=42,
    objective='binary:logistic'
)

# Model 2: Binary Classification (1 vs 2)
xgb_multi = XGBClassifier(
    n_estimators=500,
    learning_rate=0.2669112505018992,
    max_depth=5,
    random_state=42,
    objective='binary:logistic'
)

# Assuming preprocessor is defined elsewhere
model1_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Step 1: Preprocessing
    ('xgboost', xgb_binary)  # Step 2: Binary Classification
])

# Hyperparameter tuning for Model 1
param_grid = {
    'xgboost__n_estimators': [100, 300, 500],
    'xgboost__learning_rate': [0.01, 0.1, 0.2],
    'xgboost__max_depth': [3, 5, 7]
}
grid_search = GridSearchCV(model1_pipe, param_grid, cv=5, scoring='f1_micro')
grid_search.fit(X_train_full, y_train_full != 0)
best_model1_pipe = grid_search.best_estimator_

# Predict with Model 1 on train set
y_pred_model1_train = best_model1_pipe.predict(X_train_full)

# Filter data for Model 2 (predicted as "others" by Model 1 on train set)
X_other_train = X_train_full[y_pred_model1_train == 1]  # Assuming predicted class 1 means "others"
y_other_train = y_train_full[y_pred_model1_train == 1]

# Map y_other_train to binary labels (1 -> 0, 2 -> 1)
y_other_binary_train = (y_other_train == 2).astype(int)

# Train Model 2 on filtered train data
model2_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Step 1: Preprocessing
    ('xgboost', xgb_multi)  # Step 2: Binary Classification (1 vs 2)
])
model2_pipe.fit(X_other_train, y_other_binary_train)

# Predict with Model 2 on filtered train data
y_pred_model2_train = model2_pipe.predict(X_other_train)

# Final prediction based on Model 1 & 2 outputs for train set
final_preds_train = np.zeros_like(y_train_full)
final_preds_train[y_pred_model1_train == 0] = 0  # Class 0 from Model 1
final_preds_train[y_pred_model1_train == 1] = y_pred_model2_train + 1  # Class 1 or 2 from Model 2

# Evaluate the final predictions on train set
train_report = classification_report(y_train_full, final_preds_train)
print("Train Set Classification Report:\n", train_report)

# Predict with Model 1 on test set
y_pred_model1_test = best_model1_pipe.predict(X_test)

# Filter data for Model 2 (predicted as "others" by Model 1 on test set)
X_other_test = X_test[y_pred_model1_test == 1]  # Assuming predicted class 1 means "others"
y_other_test = y_test[y_pred_model1_test == 1]

# Map y_other_test to binary labels (1 -> 0, 2 -> 1)
y_other_binary_test = (y_other_test == 2).astype(int)

# Predict with Model 2 on filtered test data
y_pred_model2_test = model2_pipe.predict(X_other_test)

# Final prediction based on Model 1 & 2 outputs for test set
final_preds_test = np.zeros_like(y_test)
final_preds_test[y_pred_model1_test == 0] = 0  # Class 0 from Model 1
final_preds_test[y_pred_model1_test == 1] = y_pred_model2_test + 1  # Class 1 or 2 from Model 2

# Evaluate the final predictions on test set
test_report = classification_report(y_test, final_preds_test)
print("Test Set Classification Report:\n", test_report)

# Evaluate the model
accuracy = accuracy_score(y_test, final_preds_test)
print('Accuracy:', accuracy)

Train Set Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.60      0.71     12048
           1       0.79      0.90      0.84     79123
           2       0.82      0.69      0.75     47173

    accuracy                           0.80    138344
   macro avg       0.82      0.73      0.77    138344
weighted avg       0.81      0.80      0.80    138344

Test Set Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.45      0.55      6463
           1       0.74      0.85      0.79     42354
           2       0.76      0.63      0.69     25297

    accuracy                           0.74     74114
   macro avg       0.73      0.64      0.68     74114
weighted avg       0.74      0.74      0.74     74114

Accuracy: 0.7422349353698356


In [323]:
grid_search.best_estimator_.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('base_name',
                                                  Pipeline(steps=[('base_encoder',
                                                                   BaseNEncoder(base=3,
                                                                                cols=['land_surface_condition',
                                                                                      'geo_level_1_id',
                                                                                      'geo_level_2_id',
                                                                                      'geo_level_3_id',
                                                                                      'foundation_type',
                                                                                      'roof_type',
                                                                    

In [265]:
y_test.max

<bound method Series.max of 107180    1
257217    1
191728    0
134859    1
35271     1
         ..
249701    1
218422    1
130372    1
2939      1
6423      1
Name: damage_grade, Length: 74114, dtype: int64>

In [251]:
y.value_counts()

damage_grade
1    141177
2     84323
0     21544
Name: count, dtype: int64

In [252]:
test_data = pd.read_csv('../data/raw/test_values.csv')

In [253]:
X_test_final = test_data.building_id

In [316]:

# Load the new test data
test_data = pd.read_csv('../data/raw/test_values.csv')
#test_data = X_test
# Predict with Model 1 on new test data
y_pred_model1_test_new = best_model1_pipe.predict(test_data)

# Filter data for Model 2 (predicted as "others" by Model 1 on new test data)
X_other_test_new = test_data[y_pred_model1_test_new == 1]  # Assuming predicted class 1 means "others"

# Predict with Model 2 on filtered new test data
if len(X_other_test_new) > 0:
    y_pred_model2_test_new = model2_pipe.predict(X_other_test_new)
else:
    y_pred_model2_test_new = np.array([])

# Final prediction based on Model 1 & 2 outputs for new test data
final_preds_test_new = np.zeros(len(test_data), dtype=int)
final_preds_test_new[y_pred_model1_test_new == 0] = 0  # Class 0 from Model 1
if len(X_other_test_new) > 0:
    final_preds_test_new[y_pred_model1_test_new == 1] = y_pred_model2_test_new + 1  # Class 1 or 2 from Model 2


print(final_preds_test_new)

[2 1 1 ... 1 1 1]


In [318]:
rf_preds = pd.Series(final_preds_test_new)


In [319]:
df_concatenated = pd.concat([X_test_final, rf_preds], axis=1)
df_concatenated
df_concatenated = df_concatenated.rename(columns={0: 'damage_grade'})
df_concatenated
df_concatenated['damage_grade'] = df_concatenated['damage_grade'].replace({0: 1, 1: 2, 2: 3})
df_concatenated.to_csv('../data/processed/ATsubmission2.csv', index=False)